In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d

In [65]:
micro_data = pd.read_pickle('../divestdata/P3/micro/unified_trajectory')['mean_trajectory']
macro_data = pd.read_pickle('../divestdata/P3/aggregate/unified_trajectory')['mean_trajectory']
rep_data = pd.read_pickle('../divestdata/P3/res/unified_trajectory')['mean_trajectory']
levels=['b_d', 'phi', 'tau', 'eps']
key=[1.25, 0.5, 1.0, 0.01]

In [107]:
def l_2_distance(d1, d2, observable=None):
    
    d1_data = d1.unstack('observables')[[observable]]
    d2_data = d2.unstack('observables')[[observable]]
    
    names = d1.index.names
    print(names)
    indices = {ax: list(d1.index.levels[names.index(ax)].values) for ax in names[:2]}
    print(indices)
    output = np.zeros((len(indices[names[0]]),len(indices[names[1]])))
    
    for i, n0 in enumerate(indices[names[0]]):
        for j, n1 in enumerate(indices[names[1]]):
            levels = names[:2]
            tkey = [n0, n1]
    
            # get values and times from dataframes
            d1_slice = d1_data.xs(key=tkey, level=levels).values.T[0]
            d1_times = d1_data.xs(key=tkey, level=levels).index.values

            d2_slice = d2_data.xs(key=tkey, level=levels).values.T[0]
            d2_times = d2_data.xs(key=tkey, level=levels).index.values

            # interpolate to get values on equal timepoints
            d1_ipl = interp1d(d1_times, d1_slice)
            d2_ipl = interp1d(d2_times, d2_slice)

            samples = 200
            times = np.linspace(d1_times[0], d1_times[-1], samples)
            d1_ipl_data = d1_ipl(times)
            d2_ipl_data = d2_ipl(times)
            dd = d1_ipl_data - d2_ipl_data
            l2 = sum(dd*dd)
            output[i, j] = l2
            
    return output

In [108]:
levels=['b_d', 'eps']
key=[1.25, 0.05]


l_2_distance(micro_data.xs(key=key, level=levels), macro_data.xs(key=key, level=levels), observable='g')

['phi', 'tau', 'timesteps', 'observables']
{'phi': [0.0, 0.10000000000000001, 0.20000000000000001, 0.29999999999999999, 0.40000000000000002, 0.5, 0.59999999999999998, 0.69999999999999996, 0.80000000000000004, 0.90000000000000002, 1.0], 'tau': [0.10000000000000001, 0.31622776601683794, 1.0, 3.1622776601683795, 10.0, 31.622776601683793, 100.0, 316.22776601683796, 1000.0]}


array([[   39.548044  ,    42.74333092,   192.84918642,   509.57148251,
         1350.49690518,  1041.55545486,  1454.74866297,  2065.93794809,
         2143.80510179],
       [   57.32122269,    96.92883411,   190.10778188,   525.24021569,
         1856.68639645,   967.33294845,  1488.8492588 ,  2049.15411215,
         2130.52107222],
       [   61.94205505,   121.46709193,   282.02598094,   653.12304185,
         1795.68155297,   897.70611787,  1773.63771553,  2060.36379338,
         2144.77678309],
       [   32.72504545,    77.79363218,   239.34964857,   600.43951446,
         1055.16473325,   844.23384838,  1714.20849249,  2087.03410951,
         2150.44963831],
       [   19.60810548,   118.48968611,   262.55395168,   524.62675401,
          823.87687157,   976.33397291,  1803.93148212,  2141.01784128,
         2134.59641716],
       [   43.66542768,   223.06729407,   231.19155983,   721.74790724,
          798.07023431,  1370.85664782,  1970.69488186,  2135.85405627,
         21